GOALS: 

      CNN-LSTM/Spatial-Temporial LSTM https://towardsdatascience.com/spatial-temporal-convlstm-for-crash-prediction-411909ed2cfa
      Figure out how to break data into depth, lat and lon


#Getting Data

In [ ]:
import requests 
file_url = "https://iridl.ldeo.columbia.edu/SOURCES/.CARTON-GIESE/.SODA/.v2p2p4/ssh/data.nc"
	
r = requests.get(file_url, stream = True) 

with open("./data.nc", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block) 

import os
size = os.path.getsize('./data.nc') 
print('Size of file is', size, 'bytes')

Size of file is 1573874020 bytes


### SSH

In [10]:
import xarray as xr
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ds_ssh = xr.open_dataset("./data.nc" , decode_times=False)
#ds_ssh = xr.open_dataset("http://iridl.ldeo.columbia.edu/SOURCES/.CARTON-GIESE/.SODA/.v2p2p4/ssh/dods" , decode_times=False)

In [ ]:
def fix_calendar(ds, timevar='time'):
    if ds[timevar].attrs['calendar'] == '360':
        ds[timevar].attrs['calendar'] = '360_day'
    return ds

ds_ssh = fix_calendar(ds_ssh)
ds_ssh = xr.decode_cf(ds_ssh)
ds_ssh

<xarray.Dataset>
Dimensions:  (lat: 330, lon: 720, time: 1656)
Coordinates:
  * lon      (lon) float32 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * lat      (lat) float32 -75.25 -74.75 -74.25 -73.75 ... 88.25 88.75 89.25
  * time     (time) object 1871-01-16 00:00:00 ... 2008-12-16 00:00:00
Data variables:
    ssh      (time, lat, lon) float32 ...

In [ ]:
ds_ssh

<xarray.Dataset>
Dimensions:  (lat: 330, lon: 720, time: 1656)
Coordinates:
  * lon      (lon) float32 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * lat      (lat) float32 -75.25 -74.75 -74.25 -73.75 ... 88.25 88.75 89.25
  * time     (time) object 1871-01-16 00:00:00 ... 2008-12-16 00:00:00
Data variables:
    ssh      (time, lat, lon) float32 ...

In [ ]:
ds_ssh = ds_ssh.where(((ds_ssh.lon > 20 ) & (ds_ssh.lon < 80 ) & (ds_ssh.lat > 10) & (ds_ssh.lat <20)), drop=True)

In [ ]:
ds_ssh

<xarray.Dataset>
Dimensions:  (lat: 20, lon: 120, time: 1656)
Coordinates:
  * lon      (lon) float32 20.25 20.75 21.25 21.75 ... 78.25 78.75 79.25 79.75
  * lat      (lat) float32 10.25 10.75 11.25 11.75 ... 18.25 18.75 19.25 19.75
  * time     (time) object 1871-01-16 00:00:00 ... 2008-12-16 00:00:00
Data variables:
    ssh      (time, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan

In [ ]:
ds_ssh['ssh']

<xarray.DataArray 'ssh' (time: 1656, lat: 20, lon: 120)>
array([[[       nan,        nan,        nan, ...,        nan,
         0.47368455, 0.4705949 ],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.47322145],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.45891967],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
         0.49529132, 0.49132225],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.4916404 ],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.47177067],
...
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
         0.552477  , 0.5522664 ],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.5573779 ],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.54433674],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]]], dtype=float32)
Coordinates:
  * lon      (lon) float32 20.25 20.75 21.25 21.75 ... 78.25 78.75 79.25 79.75
  * lat      (lat) float32 10.25 10.75 11.25 11.75 ... 18.25 18.75 19.25 19.75
  * time     (time) object 1871-01-16 00:00:00 ... 2008-12-16 00:00:00
Attributes:
    pointwidth:     1.0
    units:          meters
    standard_name:  sea_surface_height
    long_name:      SEA SURFACE HEIGHT

In [ ]:
np.array(ds_ssh['ssh']).shape

(1656, 20, 120)

In [ ]:
np.array(ds_ssh['ssh'][1650][0])

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan, -0.01835824,  0.02075754,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,  0.02848718,  0.0399644 ,  0.09675951,  0.13464183,
        0.15300493,  0.20257978,  0.28198448,  0.38239264,  0.47

### Salt

In [1]:
import os
import ftplib

In [2]:
ftpURL = "ftp2.psl.noaa.gov"
ftpDirectory = "/Datasets/godas/"

In [6]:
directory_path = "./SaltData"
if not os.path.exists(directory_path): os.mkdir(directory_path)

In [3]:
ftp = ftplib.FTP(ftpURL)
ftp.login("anonymous")

'230 Login successful.'

In [4]:
ftp.cwd(ftpDirectory)

'250 Directory successfully changed.'

In [9]:
files = ftp.nlst()

In [10]:
files

['Derived',
 'README',
 'ReadMe',
 'dbss_obil.1980.nc',
 'dbss_obil.1981.nc',
 'dbss_obil.1982.nc',
 'dbss_obil.1983.nc',
 'dbss_obil.1984.nc',
 'dbss_obil.1985.nc',
 'dbss_obil.1986.nc',
 'dbss_obil.1987.nc',
 'dbss_obil.1988.nc',
 'dbss_obil.1989.nc',
 'dbss_obil.1990.nc',
 'dbss_obil.1991.nc',
 'dbss_obil.1992.nc',
 'dbss_obil.1993.nc',
 'dbss_obil.1994.nc',
 'dbss_obil.1995.nc',
 'dbss_obil.1996.nc',
 'dbss_obil.1997.nc',
 'dbss_obil.1998.nc',
 'dbss_obil.1999.nc',
 'dbss_obil.2000.nc',
 'dbss_obil.2001.nc',
 'dbss_obil.2002.nc',
 'dbss_obil.2003.nc',
 'dbss_obil.2004.nc',
 'dbss_obil.2005.nc',
 'dbss_obil.2006.nc',
 'dbss_obil.2007.nc',
 'dbss_obil.2008.nc',
 'dbss_obil.2009.nc',
 'dbss_obil.2010.nc',
 'dbss_obil.2011.nc',
 'dbss_obil.2012.nc',
 'dbss_obil.2013.nc',
 'dbss_obil.2014.nc',
 'dbss_obil.2015.nc',
 'dbss_obil.2016.nc',
 'dbss_obil.2017.nc',
 'dbss_obil.2018.nc',
 'dbss_obil.2019.nc',
 'dbss_obil.2020.nc',
 'dbss_obil.2021.nc',
 'dbss_obil.2022.nc',
 'dbss_obml.1980.nc'

In [11]:
salt_files = ftp.nlst('salt.*.nc')

In [12]:
salt_files

['salt.1980.nc',
 'salt.1981.nc',
 'salt.1982.nc',
 'salt.1983.nc',
 'salt.1984.nc',
 'salt.1985.nc',
 'salt.1986.nc',
 'salt.1987.nc',
 'salt.1988.nc',
 'salt.1989.nc',
 'salt.1990.nc',
 'salt.1991.nc',
 'salt.1992.nc',
 'salt.1993.nc',
 'salt.1994.nc',
 'salt.1995.nc',
 'salt.1996.nc',
 'salt.1997.nc',
 'salt.1998.nc',
 'salt.1999.nc',
 'salt.2000.nc',
 'salt.2001.nc',
 'salt.2002.nc',
 'salt.2003.nc',
 'salt.2004.nc',
 'salt.2005.nc',
 'salt.2006.nc',
 'salt.2007.nc',
 'salt.2008.nc',
 'salt.2009.nc',
 'salt.2010.nc',
 'salt.2011.nc',
 'salt.2012.nc',
 'salt.2013.nc',
 'salt.2014.nc',
 'salt.2015.nc',
 'salt.2016.nc',
 'salt.2017.nc',
 'salt.2018.nc',
 'salt.2019.nc',
 'salt.2020.nc',
 'salt.2021.nc',
 'salt.2022.nc']

In [13]:

#Grabs the file and writes it locally

for i in range(len(salt_files)):
   outFilename = directory_path + os.sep + salt_files[i]

   ftp.retrbinary("RETR " + salt_files[i],open(outFilename,'wb').write)

    

In [26]:
ftp.close()

In [3]:


ds_salt1 = xr.open_mfdataset(paths = '/Users/jkenglish/Desktop/NittanyAI/SaltData/salt.*.nc',combine = 'nested', concat_dim="time")


/Users/jkenglish/Downloads/anaconda3/envs/tensorflow/lib/python3.7/site-packages/xarray/backends/plugins.py:68: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [16]:
ds_salt1.to_netcdf('salt_combined.nc')

In [5]:

ds_salt2 = xr.open_dataset('salt_combined.nc',decode_times=False)


In [6]:

def fix_calendar(ds, timevar='time'):
    if ds[timevar].attrs['calendar'] == '360':
        ds[timevar].attrs['calendar'] = '360_day'
    return ds

ds_salt2 = fix_calendar(ds_salt2)
ds_salt2 = xr.decode_cf(ds_salt2)
ds_salt2




<xarray.Dataset>
Dimensions:   (level: 40, lon: 360, lat: 418, time: 506)
Coordinates:
  * level     (level) float32 5.0 15.0 25.0 ... 3.483e+03 3.972e+03 4.478e+03
  * lon       (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat       (lat) float32 -74.5 -74.17 -73.83 -73.5 ... 63.5 63.83 64.17 64.5
  * time      (time) datetime64[ns] 1980-01-01 1980-02-01 ... 2022-02-01
Data variables:
    date      (time) int32 ...
    timePlot  (time) float32 ...
    salt      (time, level, lat, lon) float32 ...
Attributes: (12/13)
    creation_date:    Sat Dec 16 20:00:00 MDT 2006
    sfcHeatFlux:      \nNote that the net surface heat flux are the total sur...
    time_comment:     The internal time stamp indicates the FIRST day of the ...
    Conventions:      COARDS
    grib_file:        godas.M.198001-12.grb
    html_REFERENCES:  http://www.cpc.ncep.noaa.gov/products/GODAS/background....
    ...               ...
    html_GODAS:       www.cpc.ncep.noaa.gov/products/GODAS
    comment:          NOTE:  THESE ARE THE BIAS CORRECTED GODAS FILES.
    title:            GODAS: Global Ocean Data Assimilation System
    References:       https://www.psl.noaa.gov/data/gridded/data.godas.html
    dataset_title:    NCEP Global Ocean Data Assimilation System (GODAS)
    history:          Created 2006/12 by Hoop\nConverted to chunked, deflated...

In [20]:
ds_salt2.keys()

KeysView(<xarray.Dataset>
Dimensions:   (level: 40, lon: 360, lat: 418, time: 506)
Coordinates:
  * level     (level) float32 5.0 15.0 25.0 ... 3.483e+03 3.972e+03 4.478e+03
  * lon       (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat       (lat) float32 -74.5 -74.17 -73.83 -73.5 ... 63.5 63.83 64.17 64.5
  * time      (time) datetime64[ns] 1980-01-01 1980-02-01 ... 2022-02-01
Data variables:
    date      (time) int32 ...
    timePlot  (time) float32 ...
    salt      (time, level, lat, lon) float32 ...
Attributes: (12/13)
    creation_date:    Sat Dec 16 20:00:00 MDT 2006
    sfcHeatFlux:      \nNote that the net surface heat flux are the total sur...
    time_comment:     The internal time stamp indicates the FIRST day of the ...
    Conventions:      COARDS
    grib_file:        godas.M.198001-12.grb
    html_REFERENCES:  http://www.cpc.ncep.noaa.gov/products/GODAS/background....
    ...               ...
    html_GODAS:       www.cpc.ncep.noaa.gov/products/G

In [21]:
np.array(ds_salt2['salt'])

array([[[[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [0.03518937, 0.03516495, 0.03513138, ..., 0.03519852,
          0.03520463, 0.03520463],
         [0.03518632, 0.0351619 , 0.03512833, ..., 0.03519242,
          0.03520158, 0.03520158],
         [0.03518327, 0.03515885, 0.03512223, ..., 0.03518937,
          0.03519852, 0.03519547]],

        [[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [0.03518937, 0.03516495, 0.03513138, ..., 0.03519852,
          0.03520768, 0.0

In [22]:
np.array(ds_salt2['salt']).shape

(506, 40, 418, 360)

In [7]:

ds_salt_sub = ds_salt2.where(((ds_salt2.lon>40 )&(ds_salt2.lon<80) & (ds_salt2.lat>10 )&(ds_salt2.lat<20)  ), drop=True)

In [8]:
ds_salt_sub

<xarray.Dataset>
Dimensions:   (time: 506, lon: 40, lat: 30, level: 40)
Coordinates:
  * level     (level) float32 5.0 15.0 25.0 ... 3.483e+03 3.972e+03 4.478e+03
  * lon       (lon) float32 40.5 41.5 42.5 43.5 44.5 ... 76.5 77.5 78.5 79.5
  * lat       (lat) float32 10.17 10.5 10.83 11.17 ... 18.83 19.17 19.5 19.83
  * time      (time) datetime64[ns] 1980-01-01 1980-02-01 ... 2022-02-01
Data variables:
    date      (time, lon, lat) float64 -2.147e+09 -2.147e+09 ... -2.147e+09
    timePlot  (time, lon, lat) float32 9.969e+36 9.969e+36 ... 2.022e+03
    salt      (time, level, lat, lon) float32 nan nan nan nan ... nan nan nan
Attributes: (12/13)
    creation_date:    Sat Dec 16 20:00:00 MDT 2006
    sfcHeatFlux:      \nNote that the net surface heat flux are the total sur...
    time_comment:     The internal time stamp indicates the FIRST day of the ...
    Conventions:      COARDS
    grib_file:        godas.M.198001-12.grb
    html_REFERENCES:  http://www.cpc.ncep.noaa.gov/products/GODAS/background....
    ...               ...
    html_GODAS:       www.cpc.ncep.noaa.gov/products/GODAS
    comment:          NOTE:  THESE ARE THE BIAS CORRECTED GODAS FILES.
    title:            GODAS: Global Ocean Data Assimilation System
    References:       https://www.psl.noaa.gov/data/gridded/data.godas.html
    dataset_title:    NCEP Global Ocean Data Assimilation System (GODAS)
    history:          Created 2006/12 by Hoop\nConverted to chunked, deflated...

In [6]:

ds_salt_sub['salt']

<xarray.DataArray 'salt' (time: 506, level: 40, lat: 30, lon: 40)>
array([[[[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [0.04186682,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan]],

        [[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
...
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan]],

        [[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan]]]], dtype=float32)
Coordinates:
  * level    (level) float32 5.0 15.0 25.0 ... 3.483e+03 3.972e+03 4.478e+03
  * lon      (lon) float32 40.5 41.5 42.5 43.5 44.5 ... 75.5 76.5 77.5 78.5 79.5
  * lat      (lat) float32 10.17 10.5 10.83 11.17 ... 18.83 19.17 19.5 19.83
  * time     (time) datetime64[ns] 1980-01-01 1980-02-01 ... 2022-02-01
Attributes:
    dataset:                  NCEP GODAS
    var_desc:                 salinity
    level_desc:               Multiple Levels
    statistic:                Monthly Mean
    parent_stat:              Individual Obs
    sub_center:               Environmental Modeling Center
    center:                   US National Weather Service - NCEP (WMC)
    long_name:                Salinity
    units:                    kg/kg
    level_indicator:          160
    gds_grid_type:            0
    parameter_table_version:  2
    parameter_number:         88
    actual_range:             [0.01370434 0.04223609]
    valid_range:              [0.  0.1]

In [7]:
np.array(ds_salt_sub['salt']).shape

(506, 40, 30, 40)

In [11]:
import pandas as pd

df_salt = pd.DataFrame()
np.array((ds_salt_sub['time'][0]))
df_salt['time'] = np.array((ds_salt_sub['time']))
df_salt['salt'] = ''
for i in range(506):
  df_salt['salt'][i] = np.array((ds_salt_sub['salt'][i]))

/Users/jkenglish/Downloads/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
df_salt

,time,salt
0,1980-01-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
1,1980-02-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
2,1980-03-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
3,1980-04-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
4,1980-05-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
...,...,...
501,2021-10-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
502,2021-11-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
503,2021-12-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
504,2022-01-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."


In [10]:
df_salt.to_csv("df_salt.csv")

### Temperatures

In [5]:

temp_files = ftp.nlst('pottmp.*.nc')

In [6]:
temp_files

['pottmp.1980.nc',
 'pottmp.1981.nc',
 'pottmp.1982.nc',
 'pottmp.1983.nc',
 'pottmp.1984.nc',
 'pottmp.1985.nc',
 'pottmp.1986.nc',
 'pottmp.1987.nc',
 'pottmp.1988.nc',
 'pottmp.1989.nc',
 'pottmp.1990.nc',
 'pottmp.1991.nc',
 'pottmp.1992.nc',
 'pottmp.1993.nc',
 'pottmp.1994.nc',
 'pottmp.1995.nc',
 'pottmp.1996.nc',
 'pottmp.1997.nc',
 'pottmp.1998.nc',
 'pottmp.1999.nc',
 'pottmp.2000.nc',
 'pottmp.2001.nc',
 'pottmp.2002.nc',
 'pottmp.2003.nc',
 'pottmp.2004.nc',
 'pottmp.2005.nc',
 'pottmp.2006.nc',
 'pottmp.2007.nc',
 'pottmp.2008.nc',
 'pottmp.2009.nc',
 'pottmp.2010.nc',
 'pottmp.2011.nc',
 'pottmp.2012.nc',
 'pottmp.2013.nc',
 'pottmp.2014.nc',
 'pottmp.2015.nc',
 'pottmp.2016.nc',
 'pottmp.2017.nc',
 'pottmp.2018.nc',
 'pottmp.2019.nc',
 'pottmp.2020.nc',
 'pottmp.2021.nc',
 'pottmp.2022.nc']

In [7]:
directory_path2 = "./TempData"
if not os.path.exists(directory_path2): os.mkdir(directory_path2)

In [8]:
for i in range(len(temp_files)):
   outFilename2 = directory_path2 + os.sep + temp_files[i]

   ftp.retrbinary("RETR " + temp_files[i],open(outFilename2,'wb').write)

In [11]:


ds_temp1 = xr.open_mfdataset(paths = '/Users/jkenglish/Desktop/NittanyAI/TempData/pottmp.*.nc',combine = 'nested', concat_dim="time")

/Users/jkenglish/Downloads/anaconda3/envs/tensorflow/lib/python3.7/site-packages/xarray/backends/plugins.py:68: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [12]:
ds_temp1.to_netcdf('temp_combined.nc')

In [11]:
ds_temp2 = xr.open_dataset('temp_combined.nc' , decode_times=False)


In [12]:
def fix_calendar(ds, timevar='time'):
    if ds[timevar].attrs['calendar'] == '360':
        ds[timevar].attrs['calendar'] = '360_day'
    return ds

ds_temp2 = fix_calendar(ds_temp2)
ds_temp2 = xr.decode_cf(ds_temp2)
ds_temp2




<xarray.Dataset>
Dimensions:   (level: 40, lon: 360, lat: 418, time: 506)
Coordinates:
  * level     (level) float32 5.0 15.0 25.0 ... 3.483e+03 3.972e+03 4.478e+03
  * lon       (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat       (lat) float32 -74.5 -74.17 -73.83 -73.5 ... 63.5 63.83 64.17 64.5
  * time      (time) datetime64[ns] 1980-01-01 1980-02-01 ... 2022-02-01
Data variables:
    date      (time) int32 ...
    timePlot  (time) float32 ...
    pottmp    (time, level, lat, lon) float32 ...
Attributes: (12/13)
    creation_date:    Sat Dec 16 20:00:00 MDT 2006
    sfcHeatFlux:      \nNote that the net surface heat flux are the total sur...
    time_comment:     The internal time stamp indicates the FIRST day of the ...
    Conventions:      COARDS
    grib_file:        godas.M.198001-12.grb
    html_REFERENCES:  http://www.cpc.ncep.noaa.gov/products/GODAS/background....
    ...               ...
    html_GODAS:       www.cpc.ncep.noaa.gov/products/GODAS
    comment:          NOTE:  THESE ARE THE BIAS CORRECTED GODAS FILES.
    title:            GODAS: Global Ocean Data Assimilation System
    References:       https://www.psl.noaa.gov/data/gridded/data.godas.html
    dataset_title:    NCEP Global Ocean Data Assimilation System (GODAS)
    history:          Created 2006/12 by Hoop\nConverted to chunked, deflated...

In [13]:
ds_temp2.keys()

KeysView(<xarray.Dataset>
Dimensions:   (level: 40, lon: 360, lat: 418, time: 506)
Coordinates:
  * level     (level) float32 5.0 15.0 25.0 ... 3.483e+03 3.972e+03 4.478e+03
  * lon       (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat       (lat) float32 -74.5 -74.17 -73.83 -73.5 ... 63.5 63.83 64.17 64.5
  * time      (time) datetime64[ns] 1980-01-01 1980-02-01 ... 2022-02-01
Data variables:
    date      (time) int32 ...
    timePlot  (time) float32 ...
    pottmp    (time, level, lat, lon) float32 ...
Attributes: (12/13)
    creation_date:    Sat Dec 16 20:00:00 MDT 2006
    sfcHeatFlux:      \nNote that the net surface heat flux are the total sur...
    time_comment:     The internal time stamp indicates the FIRST day of the ...
    Conventions:      COARDS
    grib_file:        godas.M.198001-12.grb
    html_REFERENCES:  http://www.cpc.ncep.noaa.gov/products/GODAS/background....
    ...               ...
    html_GODAS:       www.cpc.ncep.noaa.gov/products/G

In [16]:
np.array(ds_temp2['pottmp'])

array([[[[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         ...,
         [279.85764, 280.01175, 280.10178, ..., 279.27625, 279.46243,
          279.6791 ],
         [279.75082, 279.90036, 279.9904 , ..., 279.14807, 279.35254,
          279.56924],
         [279.644  , 279.78897, 279.88052, ..., 279.01837, 279.24115,
          279.45786]],

        [[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         ...,
         [279.86526, 280.01785, 280.10635, ..., 279.2854 , 279.47006,
          279.68826],
         [279.75845, 279.90646, 279.9965 , ..., 27

In [18]:
np.array(ds_temp2['pottmp']).shape

(506, 40, 418, 360)

In [14]:
ds_temp_sub = ds_temp2.where(((ds_temp2.lon>40 )&(ds_temp2.lon<80) & (ds_temp2.lat>10 )&(ds_temp2.lat<20)  ), drop=True)

In [15]:
ds_temp_sub

<xarray.Dataset>
Dimensions:   (time: 506, lon: 40, lat: 30, level: 40)
Coordinates:
  * level     (level) float32 5.0 15.0 25.0 ... 3.483e+03 3.972e+03 4.478e+03
  * lon       (lon) float32 40.5 41.5 42.5 43.5 44.5 ... 76.5 77.5 78.5 79.5
  * lat       (lat) float32 10.17 10.5 10.83 11.17 ... 18.83 19.17 19.5 19.83
  * time      (time) datetime64[ns] 1980-01-01 1980-02-01 ... 2022-02-01
Data variables:
    date      (time, lon, lat) float64 -2.147e+09 -2.147e+09 ... -2.147e+09
    timePlot  (time, lon, lat) float32 9.969e+36 9.969e+36 ... 2.022e+03
    pottmp    (time, level, lat, lon) float32 nan nan nan nan ... nan nan nan
Attributes: (12/13)
    creation_date:    Sat Dec 16 20:00:00 MDT 2006
    sfcHeatFlux:      \nNote that the net surface heat flux are the total sur...
    time_comment:     The internal time stamp indicates the FIRST day of the ...
    Conventions:      COARDS
    grib_file:        godas.M.198001-12.grb
    html_REFERENCES:  http://www.cpc.ncep.noaa.gov/products/GODAS/background....
    ...               ...
    html_GODAS:       www.cpc.ncep.noaa.gov/products/GODAS
    comment:          NOTE:  THESE ARE THE BIAS CORRECTED GODAS FILES.
    title:            GODAS: Global Ocean Data Assimilation System
    References:       https://www.psl.noaa.gov/data/gridded/data.godas.html
    dataset_title:    NCEP Global Ocean Data Assimilation System (GODAS)
    history:          Created 2006/12 by Hoop\nConverted to chunked, deflated...

In [16]:
ds_temp_sub['pottmp']

<xarray.DataArray 'pottmp' (time: 506, level: 40, lat: 30, lon: 40)>
array([[[[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         ...,
         [300.8208 ,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan]],

        [[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
...
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan]],

        [[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         ...,
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan]]]], dtype=float32)
Coordinates:
  * level    (level) float32 5.0 15.0 25.0 ... 3.483e+03 3.972e+03 4.478e+03
  * lon      (lon) float32 40.5 41.5 42.5 43.5 44.5 ... 75.5 76.5 77.5 78.5 79.5
  * lat      (lat) float32 10.17 10.5 10.83 11.17 ... 18.83 19.17 19.5 19.83
  * time     (time) datetime64[ns] 1980-01-01 1980-02-01 ... 2022-02-01
Attributes:
    dataset:                  NCEP GODAS
    var_desc:                 potential temperature
    level_desc:               Multiple Levels
    statistic:                Monthly Mean
    parent_stat:              Individual Obs
    sub_center:               Environmental Modeling Center
    center:                   US National Weather Service - NCEP (WMC)
    long_name:                Potential temperature
    units:                    K
    level_indicator:          160
    gds_grid_type:            0
    parameter_table_version:  2
    parameter_number:         13
    actual_range:             [269.0907 305.1529]
    valid_range:              [260. 310.]

In [17]:
np.array(ds_temp_sub['pottmp']).shape

(506, 40, 30, 40)

In [18]:
import pandas as pd

df_temp = pd.DataFrame()
np.array((ds_temp_sub['time'][0]))
df_temp['time'] = np.array((ds_temp_sub['time']))
df_temp['pottmp'] = ''
for i in range(506):
  df_temp['pottmp'][i] = np.array((ds_temp_sub['pottmp'][i]))

/Users/jkenglish/Downloads/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
df_temp

,time,pottmp
0,1980-01-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
1,1980-02-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
2,1980-03-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
3,1980-04-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
4,1980-05-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
...,...,...
501,2021-10-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
502,2021-11-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
503,2021-12-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."
504,2022-01-01,"[[[nan, nan, nan, nan, nan, nan, nan, nan, nan..."


In [20]:
df_temp.to_csv("df_temp.csv")

# Data Manpulation

In [ ]:
!pip install netCDF4

In [ ]:
ds = nc.Dataset('data.nc')

#Data Visulazation

In [ ]:
!pip install cartopy

In [ ]:
import cartopy.crs as ccrs

In [ ]:
days = np.arange(0,365)
# creating meshgrid from our list of spatial coordinates
lon, lat = np.meshgrid(longs,lats)
# generate a frame for each day
for i in days:
    # set the axes' spatial projection, styling, and title
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    ax.set_title('Precipitation for day '+ str(i+1))
    
    # generate the contour map
    plt.contourf(longs, lats, tp[i,], 60,
    transform=ccrs.PlateCarree())
    
    # save the figure and give a pause for visualization
    plt.savefig(f'./output/{i+1}.jpg')
    plt.pause(1)
plt.show()

In [ ]:
import PIL
image_frames = []
days = np.arange(20,80) # sub-sample day number 20 to 80
for k in days:
    new_frame = PIL.Image.open(f'./output/{k}.jpg') 
    image_frames.append(new_frame)
# save as GIF
image_frames[0].save('tp_timelapse.gif',
                     format='GIF',
                     append_images = image_frames[1: ],
                     save_all = True, 
                     duration = 100,
                     loop = 0)